In [ ]:
import os 
import sys
import numpy as np
from scipy.spatial.transform import Rotation as R 
import rospy 
import rospkg
import jupyros as jr
import pprint 
import matplotlib.pyplot as plt
from std_msgs.msg import String, Header
from geometry_msgs.msg import PoseStamped, Pose, Point, Quaternion

In [ ]:
# add catkin_ws context 
sys.path.append("/home/junting/franka_ws/devel/lib/python3.9/site-packages")

from src.lmp import *
from src.env.multimodal_env import MultiModalEnv
from src.configs.config import load_config

cfg_tabletop = load_config("perception_few_shot_gpt_3.yaml")



In [ ]:
########################################
# initialize environment
########################################
rospy.init_node('jupyter_multimodal', log_level=rospy.DEBUG)
# get package root path 
pkg_root = rospkg.RosPack().get_path('instruct_to_policy')
use_gt_2d_detections = True
use_gt_3d_bboxes = True
use_gt_planning_scene = False

if use_gt_2d_detections:
    cfg_tabletop['grounding_model']['model_name'] = 'ground_truth'
if use_gt_3d_bboxes:
    cfg_tabletop['grounding_model']['model_name'] = 'ground_truth'
    
cfg_tabletop['perception']['use_gt_2d_detections'] = use_gt_2d_detections
cfg_tabletop['perception']['use_gt_3d_bboxes'] = use_gt_3d_bboxes
cfg_tabletop['perception']['use_gt_planning_scene'] = use_gt_planning_scene


# setup environment
env = MultiModalEnv(cfg_tabletop)


In [ ]:
# env.detect_objects(["orange", "cabinet.handle_0"])
env.detect_objects(["apple", "footed_bowl"])

In [ ]:
grasp_pose = env.parse_adaptive_shape_grasp_pose("apple_0")


In [ ]:
env.open_gripper()

In [ ]:
env.publish_goal_to_marker(grasp_pose)

In [ ]:
env.close_gripper()

In [ ]:
env.attach_object("apple_0")

In [ ]:
# env.scene.visualize_3d_bboxes()
env.get_object_name_list()

In [ ]:
# get 2d visualization and matched boxes
# object_list = ["orange", "toy_bus", "basket_with_fabric_liner"]
object_list = ["apple", "footed_bowl"]
sensor_data = env.get_sensor_data()

if use_gt_2d_detections or use_gt_3d_bboxes:
    gt_bbox_3d_dict = {}
    gazebo_gt_bboxes_3d = env.get_gt_bboxes()
    for object_name in object_list:
        if object_name in gazebo_gt_bboxes_3d:
            if "." in object_name: # the object is already a part of a model
                object_id = object_name
            else:
                object_id = f"{object_name}_0"
            # FIXME: consider the case when there are multiple objects with the same name
            gt_bbox_3d_dict[object_id] = gazebo_gt_bboxes_3d[object_name]

# add the ground truth 3d bboxes to sensor data
sensor_data['bbox_3d_dict'] = gt_bbox_3d_dict

detections_list = env.grounding_model.query_2d_bbox_list(
    sensor_data=sensor_data,
    object_list=object_list
)

In [ ]:
detections_list

In [ ]:
from typing import List, Tuple
import cv2

def draw_multiview_bbox_matches(rgb_image_list: List[np.ndarray], bboxes_2d_list: List[List[np.ndarray]], 
                                matched_bboxes_idx_tuple_list: List[Tuple[np.ndarray]], object_name_list: List[str],
                                thickness=5, text_thickness=1, text_scale=0.5) -> np.ndarray:
    """
    Draw all 2D bounding box matches across different camera views with object names.
    Place all RGB images from top to bottom.
    For each tuple of matched bounding boxes across different views, draw them with the same color. Different tuples will have different colors.
    Then draw lines between the matched bounding boxes between top-to-bottom images and add object names near the bounding boxes.
    
    Args:
        rgb_image_list (List[np.ndarray]): List of RGB images.
        bboxes_2d_list (List[List[np.ndarray]]): List of lists of 2D bounding boxes.
        matched_bboxes_idx_tuple_list (List[Tuple[np.ndarray]]): List of tuples of matched 2D bounding box indices.
        object_name_list (List[str]): List of object names corresponding to each tuple in matched_bboxes_idx_tuple_list.
        thickness (int): Bounding box thickness.
        text_thickness (int): Thickness of the text.
        text_scale (float): Scale of the text.
        
    Returns:
        np.ndarray: RGB image with bounding boxes and object names.
    """
    num_views = len(rgb_image_list)
    max_width = max([rgb_image.shape[1] for rgb_image in rgb_image_list])
    total_height = sum([rgb_image.shape[0] for rgb_image in rgb_image_list])
    canvas = np.zeros((total_height, max_width, 3), dtype=np.uint8)
    
    offset = 0
    # first draw all RGB images
    for view_idx, rgb_image in enumerate(rgb_image_list):
        canvas[offset:offset + rgb_image.shape[0], :rgb_image.shape[1]] = rgb_image
        offset += rgb_image.shape[0]
        
    offset = 0
    # draw matched bounding boxes, lines, and object names
    for view_idx, rgb_image in enumerate(rgb_image_list):
        cmap = plt.cm.get_cmap('tab20')  # Choose a colormap
        for match_idx, matched_bboxes_idx_tuple in enumerate(matched_bboxes_idx_tuple_list):
            color = (np.array(cmap(match_idx % 20)[:3]) * 255).astype(int).tolist()
            if matched_bboxes_idx_tuple[view_idx] != -1:
                bbox_2d = bboxes_2d_list[view_idx][matched_bboxes_idx_tuple[view_idx]].copy()
                bbox_2d[1] += offset
                bbox_2d[3] += offset
                cv2.rectangle(canvas, (bbox_2d[0], bbox_2d[1]), (bbox_2d[2], bbox_2d[3]), color=color, thickness=thickness)
                
                # add object name text
                text_position = (bbox_2d[0], bbox_2d[1] - 10)  # adjust as needed
                cv2.putText(canvas, object_name_list[match_idx], text_position, cv2.FONT_HERSHEY_SIMPLEX, 
                            text_scale, color=color, thickness=text_thickness)
                
                if view_idx < num_views - 1 and matched_bboxes_idx_tuple[view_idx + 1] != -1:
                    next_bbox_2d = bboxes_2d_list[view_idx + 1][matched_bboxes_idx_tuple[view_idx + 1]].copy()
                    next_bbox_2d[1] += offset + rgb_image.shape[0]
                    next_bbox_2d[3] += offset + rgb_image.shape[0]

                    start_point = ((bbox_2d[0] + bbox_2d[2]) // 2, bbox_2d[3])
                    end_point = ((next_bbox_2d[0] + next_bbox_2d[2]) // 2, next_bbox_2d[1])
                    cv2.line(canvas, start_point, end_point, color=color, thickness=thickness)
        offset += rgb_image.shape[0]

    return canvas

In [ ]:
# from src.perception.utils import draw_multiview_bbox_matches

bboxes_2d_list = [
    [bboxes_2d for obj_name, bboxes_2d in detections.items()] 
    for detections in detections_list
]

matched_bboxes_idx_tuple_list = [
    [0, 0, 0],
    [1, 1, 1],
    [2, 2, 2]
]

matched_bboxes_idx_tuple_list = [
    [0, 0],
    [1, 1],
    # [2, 2]
]


canvas = draw_multiview_bbox_matches(
    rgb_image_list=sensor_data['rgb_image_list'][:2],
    bboxes_2d_list=bboxes_2d_list[:2],
    matched_bboxes_idx_tuple_list=matched_bboxes_idx_tuple_list,
    object_name_list=['orange', 'toy bus', 'basket'],
    text_thickness=2,
    text_scale=1
)

# visualize the canvas without axis
plt.figure(figsize=(10, 10))
plt.axis('off')
plt.imshow(canvas)


In [ ]:
joint_list = env.joint_prediction_model.predict(            
            {
                "object_name": 'cabinet',
                "joint_types": ["prismatic"],
            })

In [ ]:
import open3d as o3d
import numpy as np

vis_list = []


# [{'joint_position': array([-1.36175061, -0.20400387,  1.162368  ]),
#   'joint_axis': array([ 1.00000000e+00, -7.90316541e-06, -4.15183898e-09]),
#   'type': 'prismatic'},
#  {'joint_position': array([-1.38764469, -0.20400389,  0.90824719]),
#   'joint_axis': array([ 1.00000000e+00, -7.90316533e-06, -4.15313506e-09]),
#   'type': 'prismatic'},
#  {'joint_position': array([-1.38765743, -0.20400391,  0.67294999]),
#   'joint_axis': array([ 1.00000000e+00, -7.90316526e-06, -4.03956951e-09]),
#   'type': 'prismatic'},
#  {'joint_position': array([-1.38764255, -0.20400394,  0.3623576 ]),
#   'joint_axis': array([ 1.00000000e+00, -7.90316601e-06, -3.97314492e-09]),
#   'type': 'prismatic'}]

# Add 3D bounding boxes and object names
for obj_name in env.scene.object_names:
    bbox_3d = env.scene.bbox_3d_dict[obj_name]

    # Create an axis-aligned bounding box
    bbox = o3d.geometry.AxisAlignedBoundingBox(
        min_bound=np.array(bbox_3d[:3]),
        max_bound=np.array(bbox_3d[3:])
    )
    vis_list.append(bbox)
    
    vis_list.append(env.scene.scene_tsdf_full.get_mesh())

# also visualize the joints as arrows in open3d  
for joint in joint_list:
    joint_position = joint['joint_position']
    joint_axis = joint['joint_axis']
    # Create an arrow
    arrow = o3d.geometry.TriangleMesh.create_arrow(
        cylinder_radius=0.01,
        cone_radius=0.02,
        cylinder_height=0.5,
        cone_height=0.1
    )
    # Translate the arrow
    arrow.translate(joint_position)
    
    # Rotate the arrow
    o3d.geometry.get_rotation_matrix_from_axis_angle(joint_axis)
    rotaion_matrix = R.align_vectors([joint_axis], [np.array([0, 0, 1])])[0].as_matrix()
    arrow.rotate(rotaion_matrix, center=joint_position)

    # change color 
    arrow.paint_uniform_color([1, 0, 0])
        
    vis_list.append(arrow)
    
# Visualize the scene
o3d.visualization.draw_geometries(vis_list)


In [ ]:
env.get_object_name_list()

In [ ]:
import open3d as o3d
vis_list = []
for obj_name in ['apple_0', "footed_bowl_0"]:
    bbox_3d = env.scene.bbox_3d_dict[obj_name]
    # create a axis-aligned bounding box 
    bbox = o3d.geometry.AxisAlignedBoundingBox(
        min_bound=np.array(bbox_3d[:3]),
        max_bound=np.array(bbox_3d[3:])
    )
    vis_list.append(bbox)
    
vis_list.append(env.scene.scene_tsdf_full.get_mesh())

# visualize the scene
o3d.visualization.draw_geometries(vis_list)

In [ ]:
# get gt bboxes: 
gt_bboxes_dict = env.get_gt_bboxes()

In [ ]:
pprint.pprint(gt_bboxes_dict)

In [ ]:
import open3d as o3d 

# get all handle center position 
handles = ["cabinet::handle_0", "cabinet::handle_1"]

handle_centers = [
    gt_bboxes_dict[handle][0] for handle in handles
]

vis_list = []
vis_list.append(env.scene.scene_tsdf_full.get_mesh())

# visualize handle center positions as a small sphere in open3d 
for handle_center in handle_centers:
    handle_center = np.array(handle_center)
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.01)
    sphere.translate(handle_center)
    vis_list.append(sphere) 

o3d.visualization.draw_geometries(vis_list)


In [ ]:
env.add_scene_objects_to_moveit()

In [ ]:
grasp_pose = env.parse_adaptive_shape_grasp_pose('apple_0')

In [ ]:
env.planning_scene.get_known_object_names()

In [ ]:
env.objects

In [ ]:
env.open_gripper()
env.grasp(grasp_pose)


In [ ]:
env.close_gripper()
env.attach_object('apple_0')

In [ ]:
place_pose = env.parse_place_pose('apple_0', 'plate_0')
env.move_to_pose(place_pose)
env.open_gripper()
env.detach_object('apple_0')

In [ ]:
import cv2
import matplotlib.pyplot as plt

# read list of depth images, clip and replot to gray scale image
paths = [
    "/home/junting/catkin_ws/src/RoboScript/instruct_to_policy/data/benchmark/multiview_detection/depth_images/world_2_pick_and_place_camera_left.png",
    "/home/junting/catkin_ws/src/RoboScript/instruct_to_policy/data/benchmark/multiview_detection/depth_images/world_2_pick_and_place_camera_right.png",
    "/home/junting/catkin_ws/src/RoboScript/instruct_to_policy/data/benchmark/multiview_detection/depth_images/world_2_pick_and_place_camera_top.png"
]

depth_images = []
for path in paths:
    depth_image = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    depth_images.append(depth_image)

# clamp depth image to 0-5
for i, depth_image in enumerate(depth_images):
    depth_images[i] = np.clip(depth_image, 100, 5000)

# # re-scale depth image to 0-255
# for i, depth_image in enumerate(depth_images):
#     depth_images[i] = (depth_image / 5 * 255).astype(np.uint8)

# plot the depth images
# fig, axes = plt.subplots(1, len(depth_images), figsize=(12, 4))
# for i, depth_image in enumerate(depth_images):
#     axes[i].imshow(depth_image, cmap='gray')
#     axes[i].axis('off')
# plt.show()

# # plot the depth images one by one 
for depth_image in depth_images:
    plt.figure(figsize=(10, 10))
    plt.imshow(depth_image, cmap='gray')
    plt.axis('off')
    plt.show()

In [ ]:
# plot rgb images one by one 
paths = [
    "/home/junting/catkin_ws/src/RoboScript/instruct_to_policy/data/benchmark/multiview_detection/rgb_images/world_2_pick_and_place_camera_left.png",
    "/home/junting/catkin_ws/src/RoboScript/instruct_to_policy/data/benchmark/multiview_detection/rgb_images/world_2_pick_and_place_camera_right.png",
    "/home/junting/catkin_ws/src/RoboScript/instruct_to_policy/data/benchmark/multiview_detection/rgb_images/world_2_pick_and_place_camera_top.png"
]

for path in paths:
    rgb_image = cv2.imread(path)
    rgb_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(rgb_image)
    plt.axis('off')
    plt.show()